In [ ]:
!pip install boto3
import boto3

     |████████████████████████████████| 133kB 3.4MB/s 
     |████████████████████████████████| 6.7MB 10.1MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 133kB 47.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
aws_access_key_id = ""
aws_secret_access_key = ""

In [ ]:
client = boto3.client('rekognition', region_name='us-east-1',
	      aws_access_key_id = aws_access_key_id,
	      aws_secret_access_key = aws_secret_access_key)

In [ ]:
def create_collection(collection_id):
    response=client.create_collection(CollectionId=collection_id)
    print('Collection ARN: ' + response['CollectionArn'])
    print('Status code: ' + str(response['StatusCode']))

In [ ]:
# Creating Rekognition collection
create_collection("collection1")

Collection ARN: aws:rekognition:us-east-1:673982206489:collection/collection1
Status code: 200


In [ ]:
response=client.list_collections()
collection=response['CollectionIds']
print(collection)

['collection1']


In [ ]:
def add_faces_to_collection(client,bucket,photo,collection_id):
  response=client.index_faces(CollectionId=collection_id,
                              Image={'S3Object':{'Bucket':bucket,'Name':photo}},
                              ExternalImageId=photo,
                              MaxFaces=1,
                              QualityFilter="AUTO",
                              DetectionAttributes=['ALL'])

  print ('Results for ' + photo) 	
  print('Faces indexed:')						
  for faceRecord in response['FaceRecords']:
        print('  Face ID: ' + faceRecord['Face']['FaceId'])
        print('  Location: {}'.format(faceRecord['Face']['BoundingBox']))

  print('Faces not indexed:')
  for unindexedFace in response['UnindexedFaces']:
      print(' Location: {}'.format(unindexedFace['FaceDetail']['BoundingBox']))
      print(' Reasons:')
      for reason in unindexedFace['Reasons']:
          print('   ' + reason)
  return len(response['FaceRecords'])

In [ ]:
# Adding photos to Rekognition collection
bucket="owner-photos"
collection_id="collection1"
photo="Vishaal.jpg"
indexed_faces_count=add_faces_to_collection(client,bucket, photo, collection_id)
print("Faces indexed count: " + str(indexed_faces_count))

Results for Vishaal.jpg
Faces indexed:
  Face ID: 203cbffd-d693-47a5-8f65-df39cc897b16
  Location: {'Width': 0.3766229450702667, 'Height': 0.38199958205223083, 'Left': 0.3280787169933319, 'Top': 0.20352250337600708}
Faces not indexed:
Faces indexed count: 1


In [ ]:
# Listing photos from Rekognition collection
response=client.list_faces(CollectionId=collection_id)
faces = response['Faces']
for face in faces:
  print(face)

{'FaceId': '203cbffd-d693-47a5-8f65-df39cc897b16', 'BoundingBox': {'Width': 0.3766230046749115, 'Height': 0.38199999928474426, 'Left': 0.3280789852142334, 'Top': 0.20352299511432648}, 'ImageId': '8d840a8e-b8be-32e6-96cd-0ff86c57fad5', 'ExternalImageId': 'Vishaal.jpg', 'Confidence': 99.9802017211914}
{'FaceId': '35130516-5024-4ce8-8d53-21b231765e61', 'BoundingBox': {'Width': 0.10501500219106674, 'Height': 0.13894200325012207, 'Left': 0.46958601474761963, 'Top': 0.22103799879550934}, 'ImageId': '2a7e3daf-decc-354b-b33f-0c9f4c0828fb', 'ExternalImageId': 'Aditya_photo.jpeg', 'Confidence': 99.99930572509766}
